# Client Clustering 

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d


# Preprocesado y modelado
# ==============================================================================
from sklearn.preprocessing import StandardScaler


#from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# Calidad de los clusters
# ==============================================================================
from ds_utils.unsupervised import plot_cluster_cardinality, plot_cluster_magnitude, plot_magnitude_vs_cardinality
from scipy.spatial.distance import euclidean

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

### Dataset 

In [3]:
df = pd.read_csv("../data/clients_cluster.csv")
df.head()

,Unnamed: 0,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,0,5008804,1,1,1,0,427500.0,4,1,0,4,33.0,12.0,1,1,0,0,19,2.0
1,1,5008805,1,1,1,0,427500.0,4,1,0,4,33.0,12.0,1,1,0,0,19,2.0
2,2,5008806,1,1,1,0,112500.0,4,4,1,1,59.0,3.0,1,0,0,0,17,2.0
3,3,5008808,0,0,1,0,270000.0,0,4,3,1,52.0,8.0,1,0,1,1,15,1.0
4,4,5008809,0,0,1,0,270000.0,0,4,3,1,52.0,8.0,1,0,1,1,15,1.0


In [5]:
# we start the Kmean model
model = KMeans()

# we call the method KElbowVisualizer to calculate the score of silhouette
visualizer = KElbowVisualizer(model, k=(2,10), metric='silhouette')

# fiteamos el modelo
visualizer.fit(df)  

# we view the results
visualizer.show();

KeyboardInterrupt: 

### Executing our model 

In [ ]:
# we initiate the model
kmeans = KMeans(n_clusters=6)

# we fit our model
km_fit = kmeans.fit(df)

In [ ]:
# the clusters that we generate
labels = km_fit.labels_
labels

In [ ]:
# the centers of our clusters 

centroides = km_fit.cluster_centers_
centroides

In [ ]:
df["Cluster"] = labels
df.head(2)